In [1]:
import re
import time
import requests

from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)


In [3]:
# categories_urls = [
#     'https://www.amazon.in/gp/bestsellers/computers/1375424031/ref=zg_bs_nav_pc_1_pc',
#     'https://www.amazon.in/gp/bestsellers/electronics/1389396031/ref=zg_bs_nav_e_2_1389375031',
#     'https://www.amazon.in/gp/bestsellers/watches/ref=zg_bs_nav_0',
# ]

In [4]:
categories_urls = [
    'https://www.amazon.in/gp/bestsellers/computers/1375424031',
    'https://www.amazon.in/gp/bestsellers/electronics/1389396031',
    'https://www.amazon.in/gp/bestsellers/watches',
]

In [5]:
prod_title_lookup = '#productTitle'
prod_price_lookup = '#priceblock_ourprice'


In [6]:
driver.get(categories_urls[0])
body_el = driver.find_element_by_css_selector('body')
html_str = body_el.get_attribute('innerHTML')

In [7]:
# print(html_str)

In [8]:
html_obj = HTML(html=html_str)

In [9]:
# html_obj.links

In [10]:
page_links = [f'https://www.amazon.in{x}' for x in html_obj.links if x.startswith('/')]

In [11]:
# page_links_new = [x for x in page_links if 'product-reviews/' not in x]


In [12]:
len(page_links)

166

In [13]:
# first_prod_link = page_links[0]
# first_prod_link

In [14]:
# # now need to iterate over all links and see if we get the product title and price
# product_title, product_price = (None, None)
# for link in page_links:
#     time.sleep(1.2)
#     driver.get(link)
#     body_el = driver.find_element_by_css_selector('body')
#     html_str = body_el.get_attribute('innerHTML')
#     html_obj = HTML(html=html_str)
#     try:
#         product_title = html_obj.find(selector=prod_title_lookup, first=True).text
#         product_price = html_obj.find(selector=prod_price_lookup, first=True).text
#     except:
#         pass
#     print(product_title, product_price)

In [15]:
# now need to iterate over all links and see if we get the product title and price
def scrape_product_title_and_price(url, prod_title_lookup='#productTitle', prod_price_lookup='#priceblock_ourprice'):
    
    driver.get(url)
    time.sleep(1.2)
    body = driver.find_element_by_css_selector('body')
    html_string = body.get_attribute('innerHTML')
    html_object = HTML(html=html_string)
    product_title = html_object.find(selector=prod_title_lookup, first=True).text
    product_price = html_object.find(selector=prod_price_lookup, first=True).text
    return (product_title, product_price)

In [16]:

# for link in page_links_new:
#     title, price = (None, None)
#     try:
#         title, price = scrape_product_title_and_price(link)
#     except:
#         pass
#     if title != None and price != None:
#         print(link)
#         print(title)
#         print(price)
        

In [17]:
# https://www.amazon.in/HP-Pavilion-14-inch-Touchscreen-14-dh1006TU/dp/B07WFPSWVN/ref=zg_bs_1375424031_29/261-8413091-4364038?_encoding=UTF8&psc=1&refRID=M2VCJA9K4JCH85MSR9NF

# https://www.amazon.in/HP-Pavilion-14-inch-Touchscreen-14-dh1006TU/dp/B07WFPSWVN

# <base-url>/<slug>/dp/<product_id>/

In [18]:
# this is the way to name a regular exp ?P<slug> names the reg. exp [\w-]+ as slug
# w covers a-z, A-Z and 0-9, plus we add - also, so any alphanumeric string with dashes 
# is allowed. 0 or more instances !
my_url = 'https://www.amazon.in/HP-Pavilion-14-inch-Touchscreen-14-dh1006TU/dp/B07WFPSWVN/'
my_regex_pattern = r"https://www.amazon.in/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"


In [19]:
regex = re.compile(my_regex_pattern)

In [20]:
my_match = regex.match(my_url)
print(my_match)
print(my_match['slug'], my_match['product_id'])


<re.Match object; span=(0, 80), match='https://www.amazon.in/HP-Pavilion-14-inch-Touchsc>
HP-Pavilion-14-inch-Touchscreen-14-dh1006TU B07WFPSWVN


In [21]:
# on further inspection : 

regex_options = [
    r"https://www.amazon.in/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.in/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.in/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

In [22]:
def extract_productid_from_url(url):
    product_id = None
    for regex_pattern in regex_options:
        regex = re.compile(regex_pattern)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id


In [23]:
def clean_page_links(page_links=[]):
    final_page_links = []
    for link in page_links:
        product_id = extract_productid_from_url(link)
        if product_id != None:
            final_page_links.append({"url" : link, "product_id" : product_id})
    return final_page_links


            

In [27]:
cleaned_objects = clean_page_links(page_links)


In [28]:
len(page_links)

166

In [30]:
len(cleaned_objects)

50

In [ ]:
extracted_data = []
def perform_scrape(cleaned_objects=[]):
    for item in cleaned_objects:
        url = item['url']
        product_id = item['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_title_and_price(url)
        except:
            pass
        
        if title != None and price != None:
            print(url, title, price)
        # if title and price came out to be none, we can see that if we put product_data
        
        product_data = {
            'url' : url,
            'product_id' : product_id,
            'title' : title,
            'price' : price
        }
            
        
        
        